 # **Challenge Telecom-X**

## **Extracción de datos en formato JSON.**

In [27]:
import pandas as pd

# Cargar y aplanar el JSON
telecom = pd.read_json("/content/TelecomX_Data.json")
telecom_flat = pd.json_normalize(telecom.to_dict(orient='records'))

# Renombrar columnas para mayor legibilidad
telecom_flat.columns = telecom_flat.columns.str.replace('.', '_').str.lower()


In [28]:
telecom_flat.head()

,customerid,churn,customer_gender,customer_seniorcitizen,customer_partner,customer_dependents,customer_tenure,phone_phoneservice,phone_multiplelines,internet_internetservice,...,internet_onlinebackup,internet_deviceprotection,internet_techsupport,internet_streamingtv,internet_streamingmovies,account_contract,account_paperlessbilling,account_paymentmethod,account_charges_monthly,account_charges_total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [29]:
telecom_flat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerid                 7267 non-null   object 
 1   churn                      7267 non-null   object 
 2   customer_gender            7267 non-null   object 
 3   customer_seniorcitizen     7267 non-null   int64  
 4   customer_partner           7267 non-null   object 
 5   customer_dependents        7267 non-null   object 
 6   customer_tenure            7267 non-null   int64  
 7   phone_phoneservice         7267 non-null   object 
 8   phone_multiplelines        7267 non-null   object 
 9   internet_internetservice   7267 non-null   object 
 10  internet_onlinesecurity    7267 non-null   object 
 11  internet_onlinebackup      7267 non-null   object 
 12  internet_deviceprotection  7267 non-null   object 
 13  internet_techsupport       7267 non-null   objec

## **Transformación de datos**

### **Limpieza por columnas**

#### **customerID**

In [30]:
telecom_flat['customerid'] = telecom_flat['customerid'].astype(str).str.strip()

#### **churn**

In [31]:
telecom_flat['churn'] = telecom_flat['churn'].str.strip().str.title()
telecom_flat = telecom_flat[telecom_flat['churn'].isin(['Yes', 'No'])]  # quitar vacíos

#### **Gender**

In [32]:
telecom_flat['customer_gender'] = telecom_flat['customer_gender'].str.strip().str.title()

#### **SeniorCitizen**

In [33]:
telecom_flat['customer_seniorcitizen'] = telecom_flat['customer_seniorcitizen'].astype(int)

#### **Partner y Dependents**

In [34]:
telecom_flat['customer_tenure'] = pd.to_numeric(telecom_flat['customer_tenure'], errors='coerce')

#### **tenure**

In [35]:
telecom_flat['customer_tenure'] = pd.to_numeric(telecom_flat['customer_tenure'], errors='coerce')

#### **Phoneservice y Multiplelines**

In [36]:
telecom_flat['phone_phoneservice'] = telecom_flat['phone_phoneservice'].str.strip().str.title()
telecom_flat['phone_multiplelines'] = telecom_flat['phone_multiplelines'].str.strip().str.title()

#### **Servicios de internet**

In [37]:
cols_internet = [
    'internet_internetservice', 'internet_onlinesecurity', 'internet_onlinebackup',
    'internet_deviceprotection', 'internet_techsupport',
    'internet_streamingtv', 'internet_streamingmovies'
]
for col in cols_internet:
    telecom_flat[col] = telecom_flat[col].str.strip().str.title()

#### **Contract, Paperlessbilling, Paymentmethod**

In [38]:
telecom_flat['account_contract'] = telecom_flat['account_contract'].str.strip().str.title()
telecom_flat['account_paperlessbilling'] = telecom_flat['account_paperlessbilling'].str.strip().str.title()
telecom_flat['account_paymentmethod'] = telecom_flat['account_paymentmethod'].str.strip().str.title()

#### **Charges_Monthly y Charges_Total**

In [39]:
telecom_flat['account_charges_monthly'] = pd.to_numeric(telecom_flat['account_charges_monthly'], errors='coerce')
telecom_flat['account_charges_total'] = pd.to_numeric(telecom_flat['account_charges_total'], errors='coerce')

#### **Eliminar registros con datos faltantes críticos**

In [40]:
# Por ejemplo, si total charges o tenure está vacío, no sirve para análisis
telecom_flat.dropna(subset=['account_charges_total', 'customer_tenure'], inplace=True)


#### **Resetear índice**

In [41]:
telecom_flat.reset_index(drop=True, inplace=True)
telecom_flat.tail()

,customerid,churn,customer_gender,customer_seniorcitizen,customer_partner,customer_dependents,customer_tenure,phone_phoneservice,phone_multiplelines,internet_internetservice,...,internet_onlinebackup,internet_deviceprotection,internet_techsupport,internet_streamingtv,internet_streamingmovies,account_contract,account_paperlessbilling,account_paymentmethod,account_charges_monthly,account_charges_total
7027,9987-LUTYD,No,Female,0,No,No,13,Yes,No,Dsl,...,No,No,Yes,No,No,One Year,No,Mailed Check,55.15,742.90
7028,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber Optic,...,No,No,No,No,Yes,Month-To-Month,Yes,Electronic Check,85.10,1873.70
7029,9992-UJOEL,No,Male,0,No,No,2,Yes,No,Dsl,...,Yes,No,No,No,No,Month-To-Month,Yes,Mailed Check,50.30,92.75
7030,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,Dsl,...,No,Yes,Yes,No,Yes,Two Year,No,Mailed Check,67.85,4627.65
7031,9995-HOTOH,No,Male,0,Yes,Yes,63,No,No Phone Service,Dsl,...,Yes,Yes,No,Yes,Yes,Two Year,No,Electronic Check,59.00,3707.60


#### **Creación de columna "Cuentas_Diarias"**

Para crear la columna "Cuentas_Diarias" a partir de la facturación mensual (account_charges_monthly), simplemente vamos a dividir ese valor entre 30, como estimación del promedio de días por mes.

In [42]:
# Crear la columna Cuentas_Diarias como facturación mensual / 30
telecom_flat['cuentas_diarias'] = telecom_flat['account_charges_monthly'] / 30
# Ver las primeras filas para revisar la nueva columna
telecom_flat[['account_charges_monthly', 'cuentas_diarias']].head()

,account_charges_monthly,cuentas_diarias
0,65.6,2.186667
1,59.9,1.996667
2,73.9,2.463333
3,98.0,3.266667
4,83.9,2.796667


### **Estandarización y transformación de datos**

#### **1. Conversión de valores "Yes"/"No" a binarios (1 y 0)**

Aplicamos esto a todas las columnas que contienen respuestas de tipo sí/no:

In [43]:
# Lista de columnas booleanas
binarias = [
    'churn', 'customer_partner', 'customer_dependents',
    'phone_phoneservice', 'account_paperlessbilling'
]

# Añadir columnas binarias de servicios opcionales
servicios = [
    'internet_onlinesecurity', 'internet_onlinebackup',
    'internet_deviceprotection', 'internet_techsupport',
    'internet_streamingtv', 'internet_streamingmovies',
    'internet_internetservice', 'phone_multiplelines'
]

# Convertir "Yes"/"No" → 1/0
for col in binarias + servicios:
    telecom_flat[col] = telecom_flat[col].map({'Yes': 1, 'No': 0})


#### **2. Conversión de "SeniorCitizen" a tipo categórico con nombre claro**

In [44]:
telecom_flat['es_adulto_mayor'] = telecom_flat['customer_seniorcitizen'].map({1: 'Sí', 0: 'No'})

#### **3. Renombrar columnas a nombres en español más comprensibles**

In [45]:
telecom_flat.rename(columns={
    'customer_gender': 'genero',
    'customer_partner': 'tiene_pareja',
    'customer_dependents': 'tiene_dependientes',
    'customer_tenure': 'meses_contratado',
    'phone_phoneservice': 'servicio_telefonico',
    'phone_multiplelines': 'multiples_lineas',
    'internet_internetservice': 'tipo_internet',
    'internet_onlinesecurity': 'seguridad_online',
    'internet_onlinebackup': 'respaldo_online',
    'internet_deviceprotection': 'proteccion_dispositivo',
    'internet_techsupport': 'soporte_tecnico',
    'internet_streamingtv': 'tv_streaming',
    'internet_streamingmovies': 'peliculas_streaming',
    'account_contract': 'tipo_contrato',
    'account_paperlessbilling': 'factura_digital',
    'account_paymentmethod': 'metodo_pago',
    'account_charges_monthly': 'cargo_mensual',
    'account_charges_total': 'cargo_total',
    'cuentas_diarias': 'cargo_diario',
}, inplace=True)

#### **4. Estandarizar texto categórico (género, contrato, método de pago, tipo de internet)**

In [46]:
cat_cols = ['genero', 'tipo_internet', 'tipo_contrato', 'metodo_pago']

for col in cat_cols:
    telecom_flat[col] = telecom_flat[col].astype(str).str.lower().str.replace('-', ' ', regex=False).str.strip()

#### **5.Resultado final (verificación rápida)**

In [47]:
telecom_flat.head()
telecom_flat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customerid              7032 non-null   object 
 1   churn                   7032 non-null   int64  
 2   genero                  7032 non-null   object 
 3   customer_seniorcitizen  7032 non-null   int64  
 4   tiene_pareja            7032 non-null   int64  
 5   tiene_dependientes      7032 non-null   int64  
 6   meses_contratado        7032 non-null   int64  
 7   servicio_telefonico     7032 non-null   int64  
 8   multiples_lineas        6352 non-null   float64
 9   tipo_internet           7032 non-null   object 
 10  seguridad_online        5512 non-null   float64
 11  respaldo_online         5512 non-null   float64
 12  proteccion_dispositivo  5512 non-null   float64
 13  soporte_tecnico         5512 non-null   float64
 14  tv_streaming            5512 non-null   

#### ****Eliminación de  columnas que no aportan valor al análisis o a los modelos predictivos****